In [ ]:
!pip install sentence_transformers
!pip install sacrebleu

In [ ]:
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
import pickle
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import math
from nltk.translate.bleu_score import sentence_bleu
import nltk
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, InputExample, losses, util
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import sacrebleu
nltk.download('punkt')

In [ ]:
!pip install Levenshtein

In [20]:
# Function to calculate correlation coefficients between codeFunctions and codeComment using a Levenshtein distance lower than two.
def levenshtein_distance(codefunc, codesum):
	from Levenshtein import distance
	# split the strings into words
	codefunc = codefunc.lower().split()
	codesum = codesum.lower().split()
	sim_count = 0
	# count the similarity of words of codefunc to all words of codesum one by one
	for sum in codesum:
		for code in codefunc:
			if distance(sum, code) < 2:
				sim_count += 1
				break
	score = sim_count / len(codesum) if len(codesum) > 0 else 0
	return score	

def calculate_c_coefficients(dataframe):
	# Extract the columns
	code = dataframe['codeFunctions'].astype(str).values
	code_comments = dataframe['codeComment'].astype(str).values

	# Calculate the Levenshtein distance for each pair of code and comments
	levenshtein_scores = []
	for code_func, code_comment in zip(code, code_comments):
		levenshtein_score = levenshtein_distance(code_func, code_comment)
		print("c_coeff score", levenshtein_score)
		levenshtein_scores.append(levenshtein_score)
	dataframe['levenshtein_similarity'] = levenshtein_scores
	return dataframe

In [ ]:
def main():
    df_data = pd.read_csv('/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/data_with_all_metrics_scores_with-side.csv')
    print("******************* Computing c_coeff SCORES *******************\n")
    df_data = calculate_c_coefficients(df_data)
    df_data.to_csv('/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/c-coeff-test.csv')

if __name__ == '__main__':
	main()

In [4]:
import openai
from openai import OpenAI
# The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
# Set your OpenAI API key


api_key = "key"
client = OpenAI(api_key=api_key)

'''response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)'''

'response = client.embeddings.create(\n    input="Your text string goes here",\n    model="text-embedding-3-small"\n)\n\nprint(response.data[0].embedding)'

In [1]:
import pandas as pd
import numpy as np
# Load the DataFrame
df = pd.read_csv('/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/benchmark-gpt-evaluation/gpt_generated_summaries_p2.csv')
# Display the DataFrame
print(df.head())

   Unnamed: 0  id                function_name  \
0           0   1  register_component_producer   
1           1   2           process_in_batches   
2           2   3              filter_property   
3           3   4         _save_ontology_to_db   
4           4   5        _build_merge_function   

                                       codeFunctions  \
0  def register_component_producer(self, componen...   
1  def process_in_batches(tx, query, data, batch_...   
2  def filter_property(self, siid: int, piid: int...   
3  def _save_ontology_to_db(self, ontology: "Onto...   
4  def _build_merge_function(self):\n        \n\n...   

                                     originalComment  \
0  Link a component ID to its producing atom for ...   
1  Process data in batches and execute the given ...   
2                           Filter property by piid.   
3  Save graph ontology to a separate table with {...   
4  Adds tracing metadata to the control message a...   

                         

In [2]:
df = df.drop(columns=['Unnamed: 0'])
print(df.head())

   id                function_name  \
0   1  register_component_producer   
1   2           process_in_batches   
2   3              filter_property   
3   4         _save_ontology_to_db   
4   5        _build_merge_function   

                                       codeFunctions  \
0  def register_component_producer(self, componen...   
1  def process_in_batches(tx, query, data, batch_...   
2  def filter_property(self, siid: int, piid: int...   
3  def _save_ontology_to_db(self, ontology: "Onto...   
4  def _build_merge_function(self):\n        \n\n...   

                                     originalComment  \
0  Link a component ID to its producing atom for ...   
1  Process data in batches and execute the given ...   
2                           Filter property by piid.   
3  Save graph ontology to a separate table with {...   
4  Adds tracing metadata to the control message a...   

                                         codeComment    BLEU_1  ROUGE_1_P  \
0  Registers a compo

In [3]:
############### GPT Embedding Similarity Calculation ####################

import pandas as pd
import numpy as np
import openai
from openai import OpenAI

# Set your OpenAI API key
api_key = "key"
client = OpenAI(api_key=api_key)
input_file = '/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/NEW-generation_from_3_models/CS-benchmark-Python-nohw.csv'
# Load the DataFrame
df = pd.read_csv(input_file)
# Display the DataFrame
#print(df.head())

for idx, row in df.iterrows():
    code = row['target']
    code_comment = row['summary_postprocessed']
    #print("Code Functions:", code)
    #print("Code Comment:", code_comment)
    # get the embedding for the code functions using OpenAI API
    code_emb = client.embeddings.create(
        input=code,
        model="text-embedding-3-small"
        #model="text-embedding-3-large"
    )
    #print("Code Embedding:", code_emb.data[0].embedding)
    # get the embedding for the code comments using OpenAI API
    code_comment_emb = client.embeddings.create(
        input=code_comment,
        model="text-embedding-3-small"
        #model="text-embedding-3-large"
    )
    #print("Code Comment Embedding:", code_comment_emb.data[0].embedding)
    # Calculate cosine similarity
    code_embedding = code_emb.data[0].embedding
    code_comment_embedding = code_comment_emb.data[0].embedding
    similarity = np.dot(code_embedding, code_comment_embedding) / (np.linalg.norm(code_embedding) * np.linalg.norm(code_comment_embedding))
    #print("Cosine Similarity:", similarity)
    # Save the similarity score in the DataFrame
    df.at[idx, 'GPT_emb'] = similarity
# Save the updated DataFrame to a same CSV file
df.to_csv(input_file, index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/benchmark-gpt-evaluation/gpt_annotation-500.csv')
# rename column SIDE_emb to GPT_emb
df.rename(columns={'SIDE_emb': 'GPT_emb'}, inplace=True)
# save to same file
df.to_csv('/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/benchmark-gpt-evaluation/gpt_annotation-500.csv', index=False)

In [ ]:
import pandas as pd

# Read CSV
input_file = '/home/user/projects/prompt-project/SIDE_p/scripts_and_data/training_SIDE/evaluation_and_statistical_tests/NEW-generation_from_3_models/CS-benchmark-Python-nohw.csv'
df = pd.read_csv(input_file)

# Fill missing values with 0.0 in predictor columns
predictors = ["BLEU_1", "ROUGE_1_P", "ROUGE_W_R", "CodeT5_plus_CS", "BERTScore_R", 
              "SentenceBERT_CS", "InferSent_CS", "SIDE_score", "C_Coeff", "GPT_emb"]

df[predictors] = df[predictors].fillna(0.0)

# Save fixed file
df.to_csv(input_file, index=False)

print("Fixed! Missing values filled with 0.0")
print(f'Saved as: {input_file}')

In [1]:
import pandas as pd

# Load the CSV files
csv1 = pd.read_csv('/home/user/projects/prompt-project/ICPC_REPLICATION_PACKAGE/Datasets/funcom/train.csv')
csv2 = pd.read_csv('/home/user/projects/prompt-project/ICPC_REPLICATION_PACKAGE/Datasets/funcom/threshold_0_9/train_filtered_more_than_0_9.csv')

# Print number of rows
print("Rows in file1:", len(csv1))
print("Rows in file2:", len(csv2))


Rows in file1: 1184438
Rows in file2: 540170


In [1]:
import pandas as pd

def replace_target_column():
    """
    Replace the 'target' column in base-SIDE-golden-test-results@1.csv 
    with the 'target' column from CBLEU_codereval_inference_results.csv
    """
    
    # Read both CSV files
    print("Reading CSV files...")
    base_file = pd.read_csv("/home/user/projects/prompt-project/codet5+/dataset/funcom-java/inference_results-SIDE/base-SIDE-golden-test-results@1.csv")
    source_file = pd.read_csv("/home/user/projects/prompt-project/codet5+/dataset/funcom-java/inference_results-SIDE/CBLEU_codereval_inference_results.csv")
    
    print(f"Base file shape: {base_file.shape}")
    print(f"Source file shape: {source_file.shape}")
    
    # Check if both files have the same number of rows
    if len(base_file) != len(source_file):
        print(f"Warning: Files have different number of rows!")
        print(f"Base file: {len(base_file)} rows")
        print(f"Source file: {len(source_file)} rows")
        print("Using the minimum number of rows...")
        min_rows = min(len(base_file), len(source_file))
        base_file = base_file.iloc[:min_rows]
        source_file = source_file.iloc[:min_rows]
    
    # Replace the target column
    print("Replacing target column...")
    base_file['target'] = source_file['target']
    
    # Save the modified file
    output_filename = "base-SIDE-golden-test-results@1_updated.csv"
    base_file.to_csv(output_filename, index=False)
    print(f"Updated file saved as: {output_filename}")
    
    # Display first few rows to verify
    print("\nFirst 3 rows of updated file:")
    print(base_file.head(3))
    
    return base_file

# Run the function
if __name__ == "__main__":
    updated_df = replace_target_column()

Reading CSV files...
Base file shape: (230, 5)
Source file shape: (230, 4)
Replacing target column...
Updated file saved as: base-SIDE-golden-test-results@1_updated.csv

First 3 rows of updated file:
   Unnamed: 0                                             target  \
0           0  <s>Trim each element in the given string array...   
1           1  <s>Check whether the length of the given two b...   
2           2  <s>Pop an abstract type from the output frame ...   

                                  raw_predictions  correctly_predicted  \
0                 Trims the elements of an array.                    0   
1  Tells if two byte arrays have the same length.                    0   
2            Pops the last output from the stack.                    0   

   SIDE_score  
0    0.937041  
1    0.955541  
2    0.036964  
